In [93]:
import ollama
import requests
from IPython.display import Markdown, display, update_display
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from openai import OpenAI
import os
import json

In [14]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [38]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
link_system_prompt

'You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.\nYou should respond in JSON as in this example:\n{\n    "links": [\n        {"type": "about page", "url": "https://full.url/goes/here/about"},\n        {"type": "careers page": "url": "https://another.full.url/careers"}\n    ]\n}\n'

In [16]:
headers = {
 "User-Agent": 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36'
}


In [76]:
class Website:
    def __init__(self, url):
        self.url = url
        response = requests.get(self.url, headers = headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found" 
        if soup.body:
            for irrelevant in soup.body.find_all(['style', 'script', 'img', 'input']):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator = '\n', strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]
        
    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"


In [22]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)

    return user_prompt

In [ ]:
def get_links(url):
    website = Website(url)
    response = openai.responses.create(
        model="gpt-4o-mini",
        input=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ]
    )
    raw_response_text = response.output_text
    cleaned_response_text = raw_response_text.strip()
    # Remove '```json' from the start and '```' from the end
    if cleaned_response_text.startswith('```json'):
        cleaned_response_text = cleaned_response_text[len('```json'):].strip()
    if cleaned_response_text.endswith('```'):
        cleaned_response_text = cleaned_response_text[:-len('```')].strip()
    try:
        return json.loads(cleaned_response_text)
    except json.JSONDecodeError as e:
        print(f"\nJSONDecodeError: Failed to decode JSON from API response for URL: {url}")
        print(f"Error details: {e}")
        print(f"The problematic string that caused the error was: '{response.output_text}'")
        raise
    except Exception as e:
        print(f"\nAn unexpected error occurred in get_links for URL: {url}")
        print(f"Error details: {e}")
        raise # Re-raise for now to see the full traceback
    

In [78]:
url = "https://www.cursor.com/"
name = "Cursor"

In [84]:
brochure_system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [85]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [86]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [87]:
create_brochure(name, url)

Found links: {'links': [{'type': 'home page', 'url': 'https://www.cursor.com/'}, {'type': 'pricing page', 'url': 'https://www.cursor.com/pricing'}, {'type': 'features page', 'url': 'https://www.cursor.com/features'}, {'type': 'enterprise page', 'url': 'https://www.cursor.com/enterprise'}, {'type': 'blog page', 'url': 'https://www.cursor.com/blog'}, {'type': 'downloads page', 'url': 'https://www.cursor.com/downloads'}, {'type': 'students page', 'url': 'https://www.cursor.com/students'}, {'type': 'community page', 'url': 'https://www.cursor.com/community'}, {'type': 'documentation page', 'url': 'https://docs.cursor.com'}]}


# Cursor: The AI Code Editor

---

### About Us
Cursor is revolutionizing the way developers interact with code. Designed to dramatically enhance productivity, our state-of-the-art AI code editor assists developers at every step of their coding journey. Trusted by leading companies like **Johnson & Johnson**, **OpenAI**, **Stripe**, **Samsung**, **Instacart**, and many more, Cursor represents the pinnacle of intelligent software development.

---

### What We Offer
- **Smart Code Predictions**: Cursor effortlessly predicts your next code edit, allowing you to breeze through changes.
- **Natural Language Editing**: Write your code using plain instructions to make large updates efficiently.
- **Customizable Experience**: Import your favorite extensions, themes, and keybindings in just one click.
- **Strong Privacy Measures**: Our Privacy Mode ensures your code is never stored remotely, and we are SOC 2 certified for data security.

---

### Testimonials
Our users love us! Here’s what developers are saying:
> "Cursor is hands down my biggest workflow improvement in years."  
> — **Sawyer Hood, Figma**

> "With Cursor, I type ‘tab’ more than anything else. It is steps ahead of my brain."  
> — **Andrew Milich, Notion**

> "Cursor's new auto-complete is insane; it predicts what code you want based on what you're doing!"  
> — **Marc Köhlbrugge, WIP**

---

### Company Culture
At Cursor, we foster an innovative and inclusive work environment that values creativity, collaboration, and continuous learning. We believe in empowering our team members to think outside the box and always strive for improvement. The rapid pace of development means that employees can contribute ideas and see them implemented quickly, creating a dynamic workplace where everyone feels valued.

---

### Career Opportunities
We are always looking for passionate individuals to join our growing team! If you are eager to work at the intersection of AI and software development and want to make a significant impact with your work, **[Explore Careers at Cursor](#)**.

- **Open Positions**: Software Engineers, Product Managers, AI Specialists, and more.
- **Work-Life Balance**: We offer flexible work hours and a supportive remote work policy.

---

### Join Cursor Today!
Whether you’re an individual developer or part of a larger organization, discover how Cursor can transform your coding experience. **[Download Now](#)** or visit our **[Blog](#)** to see how we're leading the industry in AI-based coding tools!

For any inquiries, feel free to reach out through our **[Forum](#)** – we’d love to hear from you!

---

Transform the way you code with Cursor – your AI-powered coding partner.

In [91]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        print(chunk)
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [95]:
stream_brochure(name, url)

Found links: {'links': [{'type': 'about page', 'url': 'https://www.cursor.com/'}, {'type': 'pricing page', 'url': 'https://www.cursor.com/pricing'}, {'type': 'features page', 'url': 'https://www.cursor.com/features'}, {'type': 'enterprise page', 'url': 'https://www.cursor.com/enterprise'}, {'type': 'blog page', 'url': 'https://www.cursor.com/blog'}, {'type': 'downloads page', 'url': 'https://www.cursor.com/downloads'}, {'type': 'students page', 'url': 'https://www.cursor.com/students'}, {'type': 'community page', 'url': 'https://www.cursor.com/community'}, {'type': 'documentation', 'url': 'https://docs.cursor.com'}, {'type': 'careers page', 'url': 'https://anysphere.inc'}]}


# Cursor: The AI Code Editor Brochure

## Transform Your Coding Experience

Welcome to **Cursor**, the cutting-edge AI code editor designed to elevate your productivity and streamline your coding process. Since its inception, Cursor has helped developers achieve enhancements in their workflow, allowing engineers to focus on what truly matters: building amazing software.

### Why Choose Cursor?

- **AI-Powered Productivity**: Cursor is built to make you extraordinarily productive. With intelligent suggestions and autocomplete features, you can breeze through changes and edits like never before.
- **Familiar Environment**: Bring your favorite extensions and keybindings into Cursor with just one click, making the transition seamless and comfortable.
- **Privacy & Security**: Your code is never stored remotely, ensuring your projects remain private and secure. Cursor is SOC 2 certified to offer peace of mind.

### Trusted by Industry Leaders

Cursor is used by engineers at top companies including:

- **Johnson & Johnson**
- **OpenAI**
- **Stripe**
- **Samsung**
- **Instacart**
- **Shopify**

Our users rave about Cursor:

> "Cursor is hands down my biggest workflow improvement in years." - *Sawyer Hood, Figma*

> "It's amazing having an AI pair programmer." - *Ben Bernard, Instacart*

### Features That Boost Your Workflow

- **Predictive Editing**: Cursor anticipates your next move, suggesting relevant edits and changes quickly.
- **Natural Language Coding**: Write whole functions or classes with simple instructions and watch Cursor make the necessary adjustments for you.
- **Frequent Updates**: Our team consistently improves Cursor, adding new features to enhance the user experience.

### Company Culture

At Cursor, we foster a culture of innovation and collaboration. Our work environment is built on mutual respect and the shared goal of enhancing the developer experience. We value creativity, passion for technology, and the drive to solve complex problems. Join us, and become part of our thriving community that celebrates the fusion of AI and coding.

### Career Opportunities

We are on the lookout for talented and motivated individuals who are ready to contribute to the future of coding. Explore exciting career opportunities at Cursor where your ideas can shape the development landscape.

- **Current Openings**: Check our Careers page for the latest job opportunities.
- **Work Environment**: Be part of a dynamic team that values growth, learning, and professional development.

### Get Started with Cursor Today

Discover how Cursor can revolutionize your coding experience. Download the app for **MacOS** and enjoy seamless coding powered by AI.

#### Connect With Us

Stay updated by visiting our Blog and Forum. Join the conversation, share your insights, and learn from fellow developers.

**[Download Cursor](#)** | **[Explore Features](#)** | **[Join Our Team](#)**

---

Elevate your coding journey with **Cursor**, where technology meets creativity. Experience the future of programming today!